In [1]:
import scipy.io as sio
import numpy as np
import mat73
import center_algorithms as ca
import matplotlib.pyplot as plt
import k_means as km
import seaborn as sns
import pandas as pd
from sklearn.manifold import MDS


In [2]:
n_its= 20
seed = 0

labels_raw = sio.loadmat('./data/kmeans_action_labels.mat')['kmeans_action_labels']

labels_true = [l[0][0] for l in labels_raw['labels'][0][0]]
# labelidxs =labels_raw['labelidxs'][0][0][0]


raw_data = mat73.loadmat('./data/kmeans_pts.mat')

X = [t[0] for t in raw_data['Data']['gr_pts']]

print(len(X))
X[0].shape



601


(1024, 48)

In [3]:
label_counts = {}
for l in labels_true:
    label_counts[l] = len(np.where(np.array(labels_true) == l)[0])
label_counts

{'push-cart': 20,
 'clap-group': 41,
 'ride-bike': 17,
 'swing': 3,
 'follow': 32,
 'flap': 11,
 'loiter': 185,
 'bend': 27,
 'run': 24,
 'point-group': 33,
 'cart-motion': 9,
 'stand': 5,
 'pickup': 27,
 'gesture': 39,
 'run-group': 4,
 'walk-rifle': 7,
 'loiter-group': 117}

In [4]:
# #synthetic test
# def distance_matrix(X, C, opt_type = 'sine'):
#     n = len(X)
#     m = len(C)
#     Distances = np.zeros((m, n))
#     for i in range(m):
#         for j in range(n):
#             Distances[i,j] = np.linalg.norm(C[i]-X[j])
#     return Distances

# X = []
# for i in range(10):
#     X.append(np.array([np.random.uniform(0, .5), np.random.uniform(0, .5)]))
#     X.append(np.array([np.random.uniform(1, 1.5), np.random.uniform(1, 1.5)]))
    
# for x in X:
#     plt.scatter(x[0],x[1], color = 'b')

# for x in X:
#     plt.scatter(x[0],x[1], color = 'b')
# for c in centers:
#     plt.scatter(c[0],c[1], color = 'y')

In [5]:

'''
datapoints rows and centers columns

sine
cosine
sine_sq
'''
def distance_matrix(X, C, opt_type = 'sine'):
    n = len(X)
    m = len(C)
    Distances = np.zeros((m,n))

    for i in range(m):
        for j in range(n):
            Distances[i,j] = ca.calc_error_1_2([C[i]], X[j], opt_type)
            
    return Distances

def cluster_purity(X, centers, opt_type):
    #calculate distance matrix
    d_mat = distance_matrix(X, centers, opt_type)

    #find the closest center for each point
    index = np.argmin(d_mat, axis = 0)
    
    count = 0
    for i in range(len(centers)):
        idx = np.where(index == i)[0]
        cluster_labels = [labels_true[i] for i in idx]
        most_common_label = max(set(cluster_labels), key = cluster_labels.count)
        count += cluster_labels.count(most_common_label)

    return count/len(X)

In [6]:

def lbg_subspace(X, n_centers = 17, opt_type = 'sine', n_its = 10):
    n_centers 
    n_pts = len(X)
    error = 1
    opt_type 
    new_distortion = 0
    n_its = 10
    r = 48

    #init centers
    centers = []
    for i in range(n_centers):
        centers.append(X[np.random.randint(n_pts)])


    errors = []
    distortions = []
    while error > .00001:

        old_distortion = new_distortion

        m = len(centers)

        #calculate distance matrix
        d_mat = distance_matrix(X, centers, opt_type)

        #find the closest center for each point
        index = np.argmin(d_mat, axis = 0)

        centers = []
        for c in range(m):
            idx = np.where(index == c)[0]
            if len(idx) > 0:
                if opt_type == 'sine_sq':
                    centers.append(ca.flag_mean([X[i] for i in idx], r, fast = False))
                else:
                    centers.append(ca.irls_flag([X[i] for i in idx], r, n_its, opt_type, opt_type)[0])
        #         centers.append(np.mean([X[i] for i in idx], axis = 0))

        new_distortion = np.sum(d_mat[index])

        error = np.abs(new_distortion - old_distortion)/new_distortion
        errors.append(error)
        print(error)

    return centers, error





In [ ]:
print('sin start')
centers_sin, error_sin = lbg_subspace(X, n_centers = 4, opt_type = 'sine', n_its = 10)
print('cos start')
centers_cos, error_cos = lbg_subspace(X, n_centers = 4, opt_type = 'cossine', n_its = 10)
print('flg start')
centers_flg, error_flg = lbg_subspace(X, n_centers = 4, opt_type = 'sine_sq', n_its = 10)

sin start
